In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
import json
import os
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from langchain.memory import ConversationBufferMemory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables.history import RunnableWithMessageHistory, RunnableLambda
from operator import itemgetter

# OpenAI LLM 설정
llm = ChatOpenAI(model="gpt-4o", temperature=0.5)
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

VECTOR_STORE_DIR = "C:\Project\real_set\data\vector_store"

# 사용자 입력의 의도를 분류하는 프롬프트
intent_prompt = PromptTemplate(
    input_variables=["query"],
    template="""
    사용자의 입력을 다음 5가지 중 하나로 분류하세요:
    1-1. 웹툰 추천
    1-2. 웹소설 추천
    1-3. 웹툰, 웹소설 함께 추천
    (예시)
    - "재밌는 무협 추천해줘." -> "다음과 같은 웹툰들을 추천드립니다. 웹툰과 웹소설 중 선택해 주시면 더 자세히 추천드릴 수 있습니다."

    2-1. 일상 대화 -> 추천과 연관 지을 수 있음
    (예시)
    - "아 회사 가기 싫다." -> "출근은 언제나 힘들죠. 직장인 일상물을 웹툰을 추천드릴게요."
    - "어우 졸려." -> "졸리시군요! 지루함을 달랠만 한 웹툰을 추천드릴게요."
    - "햄버거 너무 맛있다." -> "맛있는 햄버거를 드셨나보군요!. 먹음직스러운 음식이 나오는 웹툰을 추천드릴게요."
    - "아 주식 개망했다." -> "쉽지 않죠 ㅎㅎ 평범했던 주인공이 재벌급으로 성장하는 웹툰을 추천드릴게요. 다시 자신감이 생길거에요."

    2-2. 일상 대화 -> 추천과 연관 지을 수 없음
    - "너는 진보야 보수야" -> "죄송합니다. 저는 정치적 견해를 가지고 있지 않습니다. 다른 질문을 주시면 웹툰, 웹소설을 추천드릴게요."
    - "20*30-10는 뭐야?" -> "590입니다. 웹툰, 웹소설과 관련된 질문을 주시면 추천해드리겠습니다."

    사용자 입력: {query}
    의도 번호:
    """
)
intent_chain = LLMChain(llm=llm, prompt=intent_prompt)

def load_vector_store(store_name):
    store_path = os.path.join(VECTOR_STORE_DIR, store_name)
    
    if not os.path.exists(store_path):
        print(f"벡터 스토어가 존재하지 않습니다: {store_path}")
        return None  # 벡터 스토어가 없으면 None 반환
    
    return Chroma(persist_directory=store_path, embedding_function=embedding_model)  

# 벡터 스토어 불러오기 
webtoon_vector_store = load_vector_store("webtoon_vector_store")
webnovel_vector_store = load_vector_store("webnovel_vector_store")
combined_vector_store = load_vector_store("total_vector_store") 

# 벡터 스토어가 None이면 오류 방지
if webtoon_vector_store is None or webnovel_vector_store is None or combined_vector_store is None:
    print("일부 벡터 스토어가 존재하지 않습니다. 먼저 벡터 스토어를 생성하세요.")
    exit()

# 사용자 입력을 처리하는 함수
def handle_user_query(user_query):
    try:
        intent_response = intent_chain.invoke({"query": user_query})
        intent_number = intent_response["text"].strip()  
    except Exception as e:
        return f"의도 분석 중 오류 발생: {str(e)}"

    print(f"\n[의도 파악] 사용자 입력: {user_query}")
    print(f"[분류된 의도]: {intent_number}\n")

    # 의도에 따라 적절한 벡터 스토어 선택
    if intent_number in ["1-1", "2-1"]:
        retriever = webtoon_vector_store.as_retriever()
    elif intent_number == "1-2":
        retriever = webnovel_vector_store.as_retriever()
    elif intent_number == "1-3":
        retriever = combined_vector_store.as_retriever()
    else:
        return "적절한 검색 데이터를 찾을 수 없습니다."

    # 검색 실행
    docs = retriever.get_relevant_documents(user_query)

    # 검색 결과가 없을 경우 처리
    if not docs:
        print("\n[검색 결과 없음]\n")
        return "관련된 정보를 찾을 수 없습니다. 다시 질문해 주세요."

    print(f"\n[검색된 문서 개수]: {len(docs)}")
    for idx, doc in enumerate(docs[:3]):  # 상위 3개만 출력
        print(f"\n[문서 {idx+1}]:\n{doc.page_content}\n")

    # 검색 결과를 LLM에 전달하여 최종 응답 생성
    context_data = "\n".join([doc.page_content for doc in docs])

    if not context_data.strip():
        context_data = "추천 데이터를 찾을 수 없습니다."

    final_prompt = f"사용자의 의도: {intent_number}\n입력된 질문: {user_query}\n추천 데이터:\n{context_data}"
    
    try:
        final_response = llm.invoke(final_prompt) 
    except Exception as e:
        final_response = f"LLM 응답 생성 중 오류 발생: {str(e)}"

    return final_response

memory = ConversationBufferMemory()
parser = StrOutputParser()

# 사용자 질문 입력
query = input("질문을 입력하세요: ").strip()
if not query:
    print("유효한 질문을 입력하세요.")
    exit()

response = handle_user_query(query)

print("\n=== 최종 응답 ===\n")
print(response)
print("\n=================")


C:\Users\Playdata\AppData\Local\Temp\ipykernel_23328\817426989.py:15: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4o", temperature=0.5)
C:\Users\Playdata\AppData\Local\Temp\ipykernel_23328\817426989.py:16: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
C:\Users\Playdata\AppData\Local\Te

eal_set\dataector_store\webtoon_vector_store
eal_set\dataector_store\webnovel_vector_store
eal_set\dataector_store\total_vector_store
일부 벡터 스토어가 존재하지 않습니다. 먼저 벡터 스토어를 생성하세요.

[의도 파악] 사용자 입력: 요즘 10대들은 뭘 많이 봐?
[분류된 의도]: 사용자 입력 "요즘 10대들은 뭘 많이 봐?"는 심층적 분류가 필요합니다. 따라서 의도 번호는 3-1입니다. 유도 질문으로 "10대들이 좋아하는 웹툰이나 웹소설을 찾고 계신가요?"라고 물어볼 수 있습니다.


=== 최종 응답 ===

적절한 검색 데이터를 찾을 수 없습니다.



: 